In [ ]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# In[2]:
torch.set_num_threads(8)
torch.get_num_threads()

#### Load Data

In [ ]:
# In[3]:
df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)
print(df_train.shape, df_test.shape)
# In[4]:
df_train.info()
# In[5]:


In [ ]:
df_model = df_train

In [ ]:
fea_cols = ['layer_1', 'layer_2', 'layer_3', 'layer_4']
fea_cols = [c for c in df_train.columns if 'layer_' not in c]
df_train['layer_sum'] = df_train[['layer_1', 'layer_2', 'layer_3', 'layer_4']].sum(axis=1)
y_cols = ['layer_sum']

In [ ]:
df_train[fea_cols + y_cols]

In [ ]:
model = nn.Linear(in_features=len(fea_cols), out_features=len(y_cols), bias=True).to(device)
# criterion = nn.MSELoss().to(device)
criterion = nn.L1Loss().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [ ]:
x = torch.from_numpy(df_train[fea_cols].values).float().to(device)
y = torch.from_numpy(df_train[y_cols].values).float().to(device)


In [ ]:
for step in range(100000):
    prediction = model(x)
    loss = criterion(input=prediction, target=y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
loss.item()

In [ ]:
x_test = torch.from_numpy(df_train[:50][fea_cols].values).float().to(device)
y_test = torch.from_numpy(df_train[:50][y_cols].values).float().to(device)

In [ ]:
y_pred = model(x_test)
print(torch.cat([y_pred, y_test], axis=1))